In [1]:
#used to import data
import pandas as pd
#Evaluates the model
from sklearn.metrics import log_loss
import time #just used to see how long the model takes to run
from pytorch_tabnet.tab_model import TabNetRegressor

In [2]:
df_11 = pd.read_csv('data/pbp/ready_to_model/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/17_18_pbp_cleaned.csv')
df_19 = pd.read_csv('data/pbp/ready_to_model/18_19_pbp_cleaned.csv')
df_20 = pd.read_csv('data/pbp/ready_to_model/19_20_pbp_cleaned.csv')
df_21 = pd.read_csv('data/pbp/ready_to_model/20_21_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17,df_18,df_19,df_20,df_21])

In [3]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop(['is_goal'], axis=1).to_numpy()
    y = df.is_goal.to_numpy()
    
    return X,y

In [4]:
X,y = get_X_y(df)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=101)

In [6]:
#X_train.values
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [7]:
reg = TabNetRegressor()
reg.fit(X_train, y_train, eval_metric = ['logloss'], max_epochs = 50, patience=10)

Device used : cpu
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.0581  |  0:00:52s
epoch 1  | loss: 0.05279 |  0:01:43s
epoch 2  | loss: 0.05184 |  0:02:30s
epoch 3  | loss: 0.05146 |  0:03:20s
epoch 4  | loss: 0.0513  |  0:04:08s
epoch 5  | loss: 0.05122 |  0:04:59s
epoch 6  | loss: 0.05115 |  0:05:44s
epoch 7  | loss: 0.0511  |  0:06:30s
epoch 8  | loss: 0.05108 |  0:07:18s
epoch 9  | loss: 0.05103 |  0:08:01s
epoch 10 | loss: 0.05103 |  0:08:44s
epoch 11 | loss: 0.05101 |  0:09:33s
epoch 12 | loss: 0.051   |  0:10:17s
epoch 13 | loss: 0.05098 |  0:10:59s
epoch 14 | loss: 0.05095 |  0:11:56s
epoch 15 | loss: 0.05091 |  0:12:50s
epoch 16 | loss: 0.0509  |  0:13:38s
epoch 17 | loss: 0.05087 |  0:14:27s
epoch 18 | loss: 0.05089 |  0:15:17s
epoch 19 | loss: 0.05086 |  0:16:08s
epoch 20 | loss: 0.05084 |  0:16:55s
epoch 21 | loss: 0.05086 |  0:17:39s
epoch 22 | loss: 0.05082 |  0:18:34s
epoch 23 | loss: 0.05084 |  0:19:32s
epoch 24 | loss: 0.05

In [8]:
preds = reg.predict(X_test)
print(log_loss(y_test, preds))

0.19770059413374053


In [ ]:
#after 10 epochs logloss = .197
#after 10 epochs logloss = .197